Datset 

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('venv/explanation_dataset.csv')

# Split into layperson and expert datasets
layperson_df = df[df['target_audience'] == 'layperson']
expert_df = df[df['target_audience'] == 'expert']

# Save the datasets to separate CSV files
layperson_df.to_csv('layperson_explanation_dataset.csv', index=False)
expert_df.to_csv('expert_explanation_dataset.csv', index=False)

# Verify the structure of the DataFrames
print("Layperson Dataset:")
print(layperson_df)
print("\nExpert Dataset:")
print(expert_df)

In [ ]:
def create_in_context_prompt(examples, new_example):
    prompt = "Here are some examples of explanations and their evaluations:\n\n"
    for ex in examples:
        prompt += f"Explanation: {ex['explanation']}\n"
        prompt += f"Target Audience: {ex['target_audience']}\n"
        prompt += f"Confidence Score: {ex['confidence_score']}\n\n"
    prompt += "Now, evaluate the following explanation:\n\n"
    prompt += f"Explanation: {new_example}\n"
    prompt += "Evaluate the explanation and provide a confidence score and feedback."
    return prompt

In [ ]:
import pandas as pd
import openai

openai.api_key = 'sk-proj-eCKWxtFXUDb0yThKFtx1T3BlbkFJXvgHaR4d455KUPYGVi4L'

# Function to create in-context prompt


def create_in_context_prompt(examples, new_example):
    prompt = "Here are some examples of explanations and their evaluations:\n\n"
    for _, ex in examples.iterrows():
        prompt += f"Explanation: {ex['explanation']}\n"
        prompt += f"Target Audience: {ex['target_audience']}\n"
        prompt += f"Confidence Score: {ex['confidence_score']}\n\n"
    prompt += "Now, evaluate the following explanation:\n\n"
    prompt += f"Explanation: {new_example}\n"
    prompt += "Evaluate the explanation and provide a confidence score and feedback."
    return prompt

# Function to validate explanation with GPT


def validate_explanation_with_gpt(explanation, examples):
    prompt = create_in_context_prompt(examples, explanation)
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert assistant providing evaluation and feedback."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=200,
        temperature=0.7,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response['choices'][0]['message']['content']


# Load examples from CSV
examples_df = pd.read_csv('explanation_dataset.csv')

# Select a subset of examples to use for the in-context learning prompt
selected_examples = examples_df.sample(n=5)

# New explanation to evaluate
new_explanation = "In the second interval, the value of the opponent's offer has to be at least as large as a set minimum value or another calculated value."

# Get feedback
feedback = validate_explanation_with_gpt(new_explanation, selected_examples)
print(feedback)

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

# Define the custom dataset class


class ExplanationDataset(Dataset):
    def __init__(self, dataframe):
        self.labels = dataframe['confidence_score'].values
        self.texts = dataframe['explanation'].values
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx], truncation=True, padding='max_length', max_length=512)
        item = {key: torch.tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

# Function to train a model


def train_model(df, model_name):
    # Split the dataset
    train_size = int(0.8 * len(df))
    val_size = len(df) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        ExplanationDataset(df), [train_size, val_size])

    # Load BERT model
    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased', num_labels=1)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f'./results_{model_name}',
        num_train_epochs=3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=f'./logs_{model_name}',
        logging_steps=10,
        evaluation_strategy="steps",
        eval_steps=50
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    # Train the model
    trainer.train()
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    # Save the model and tokenizer
    model.save_pretrained(f'bert-finetuned-{model_name}')
    tokenizer.save_pretrained(f'bert-finetuned-{model_name}')


# Load the datasets
df_layperson = pd.read_csv('layperson_explanation_dataset.csv')
df_expert = pd.read_csv('expert_explanation_dataset.csv')

# Train the model for layperson explanations
train_model(df_layperson, 'layperson')

# Train the model for expert explanations
train_model(df_expert, 'expert')

In [10]:
from dotenv import load_dotenv
import os
import openai

load_dotenv()  # Load environment variables from .env file
openai.api_key = os.getenv("OPENAI_API_KEY")

In [11]:
import os
import openai
import torch
from dotenv import load_dotenv
from transformers import BertTokenizer, BertForSequenceClassification, TextClassificationPipeline

load_dotenv() 
openai.api_key = os.getenv("OPENAI_API_KEY")



# Generating enriched explanation from mathematical sentence
domain = (
    "two agents representing two people living together while organizing a party negotiate over 6 issues: "
    "the food type, drinks type, location, type of invitations, music, and the clean-up service. Each issue "
    "further consists of 3 to 5 values, resulting in a domain with 3072 total possible outcomes."
)

def enrich_explanation(sentence):
    
    prompt = (
        f"Provide a clear and concise explanation of the following statement in just 1 or 2 lines. Consider the domain context:\n\n"
        f"{domain}\n\n"
        f"Statement: {sentence}\n\nExplanation:"
    )

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You're an expert assistant who provides clear and concise explanation"},
            {"role": "user", "content": prompt}
        ],
        max_tokens=400,
        temperature=0.7,
        top_p=0.9,
        frequency_penalty=0.0,
        presence_penalty=0.0,
    )

    enriched_explanation = response['choices'][0]['message']['content'].strip()
    return enriched_explanation


sentence = r"Ensures \(U_u(\omega_t^o) \) meets either a calculated statistical value or a specified minimum utility requirement in the initial interval \( [0.000, 0.0361) \)"
enriched_sentence = enrich_explanation(sentence)
print(f"Enriched Explanation:\n{enriched_sentence}\n")

Enriched Explanation:
This statement is ensuring that the utility function \(U_u(\omega_t^o) \), which represents the satisfaction or benefit of an agent's choice in the negotiation, meets either a statistically calculated value or a pre-determined minimum utility requirement within the initial range of \( [0.000, 0.0361) \).



In [ ]:
def prompt_layperson(enriched_sentence):
    return (
        "Your task is to explain the following mathematical statement in very simple terms, suitable for someone without any technical background. The explanation should be direct, clear, and within 30 words. Avoid using any jargon, complex terms, or additional examples. Focus on making the statement understandable as it is.\n\n"
        f"**Mathematical Statement:**\n{enriched_sentence}\n\n"
        "**Your Task:**\n"
        "Based on the mathematical statement provided, generate a clear and simple explanation suitable for a layperson, within 30 words."
    )

In [ ]:
# Enrich the mathematical sentence
sentence = r"Ensures \(U_u(\omega_t^o) \) meets either a calculated statistical value or a specified minimum utility requirement in the initial interval \( [0.000, 0.0361) \)"
enriched_sentence = enrich_explanation(sentence)

# Generate custom explanation for layperson
layperson_output = custom_explanation(
    enriched_sentence, 'layperson', prompt_layperson)
print("Layperson Explanation Output:\n", layperson_output)

In [ ]:
def prompt_layperson(enriched_sentence):
    return (
        "Your task is to explain the following mathematical statement in very simple terms, suitable for someone without any technical background. The explanation should be clear, concise, and within 30 words. Avoid using any jargon or complex terms. Refer to the examples below for the style of explanation:\n\n"
        f"**Mathematical Statement:**\n{enriched_sentence}\n\n"
        "**Examples of Clear Explanations for a Layperson:**\n"
        "1. The final price should match the average market price or include a discount, ensuring it is fair and competitive.\n"
        "2. In the first phase, the plan should improve basic features to be at least as good as a standard option.\n"
        "3. The service package should meet a basic quality level or reach a specific customer satisfaction score to ensure a good experience.\n"
        "4. The initial budget must be large enough to cover all estimated costs and any additional expenses.\n\n"
        "**Your Task:**\n"
        "Based on the mathematical statement provided, generate a clear and simple explanation suitable for a layperson, within 50 words."
    )


# Prompt for expert explanation


def prompt_expert(enriched_sentence):
    return (
        "Provide a detailed and technical explanation of the following mathematical statement for a domain expert. The explanation should be within 50 words. Refer to the examples below for the style of explanation:\n\n"
        f"**Mathematical Statement:**\n{enriched_sentence}\n\n"
        "**Explanation for Domain Expert:**\n"
        "1. During the second interval [0.0361, 1.000], the utility of the opponent's offer \( U_u(\omega_t^o) \) must exceed the higher of a predefined threshold \( u \) or the quantile function \( U_{\Omega^o_t} \) at a specific time-dependent point.\n"
        "2. The initial evaluation phase requires the service package value \( V_s \) to surpass the minimum quality benchmark or meet a defined satisfaction threshold to ensure compliance with service standards.\n"
        "3. The order quantity \( Q_s \) must align with the highest value between the minimum stock level and a demand forecast quantile to optimize inventory management during the initial stocking phase.\n\n"
        "**Your Task:**\n"
        "Provide a similar style explanation suitable for an expert, within 50 words."
    )

In [ ]:
def custom_explanation(sentence, target_audience, prompt_func, confidence_score=None, max_tokens=400, temperature=0.6, top_p=0.7, frequency_penalty=0.0, presence_penalty=0.0):
    # Generate the initial prompt based on the target audience
    prompt = prompt_func(enriched_sentence)

    # If confidence_score is provided, generate feedback
    if confidence_score is not None:
        feedback = generate_feedback(
            enriched_sentence, confidence_score, target_audience)
        prompt += f"\n\nFeedback for Improvement:\n{feedback}\n\nRefine the explanation based on the feedback."
    else:
        feedback = None

    # Use OpenAI's API to get the explanation
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert assistant. Your task is to provide clear and concise explanations for the specified audience."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
    )

    # Extract the custom explanation from the response
    custom_explanation = response['choices'][0]['message']['content'].strip()

    # Check if the response is close to the token limit and add a note if it is
    if len(custom_explanation) >= max_tokens - 20:
        custom_explanation += " (response cut off, please refine or increase token limit)"

    return custom_explanation



In [ ]:
# Example usage
layperson_output = custom_explanation(
    enriched_sentence, 'layperson', prompt_layperson)

expert_output = custom_explanation(enriched_sentence, 'expert', prompt_expert)

print("Layperson Explanation Output:\n", layperson_output)
print("\nExpert Explanation Output:\n", expert_output)

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, TextClassificationPipeline

# Load the fine-tuned BERT models and tokenizers for each audience
layperson_model = BertForSequenceClassification.from_pretrained(
    'bert-finetuned-layperson')
layperson_tokenizer = BertTokenizer.from_pretrained('bert-finetuned-layperson')

expert_model = BertForSequenceClassification.from_pretrained(
    'bert-finetuned-expert')
expert_tokenizer = BertTokenizer.from_pretrained('bert-finetuned-expert')

# Function to validate an explanation using the appropriate model and tokenizer


def validate_explanation(explanation, target_audience, max_length=512):
    if target_audience == 'layperson':
        model = layperson_model
        tokenizer = layperson_tokenizer
    elif target_audience == 'expert':
        model = expert_model
        tokenizer = expert_tokenizer
    else:
        raise ValueError(
            "Invalid target audience. Choose either 'layperson' or 'expert'.")

    model.eval()
    inputs = tokenizer(explanation, return_tensors="pt",
                       truncation=True, padding='max_length', max_length=max_length)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    confidence_score = torch.sigmoid(logits).item()
    return confidence_score

# Function to generate feedback based on the confidence score and target audience


def generate_feedback(explanation, confidence_score, target_audience):
    feedback = ""

    if target_audience == 'layperson':
        if confidence_score < 0.4:
            feedback = "The explanation is too complex and difficult for a layperson to understand. Simplify the language, remove technical terms, and use more relatable examples."
        elif 0.4 <= confidence_score < 0.7:
            feedback = "The explanation is somewhat clear but could be improved. Consider simplifying the language further and ensuring it is more engaging for a layperson."
        elif confidence_score >= 0.7:
            feedback = "The explanation is clear and easy to understand. It's well-suited for a layperson, but consider making it even more engaging or concise."
    elif target_audience == 'expert':
        if confidence_score < 0.4:
            feedback = "The explanation lacks the necessary technical depth and detail for an expert. Include more precise terms, context, and relevant details to improve it."
        elif 0.4 <= confidence_score < 0.7:
            feedback = "The explanation is somewhat detailed but could benefit from additional technical depth. Ensure all relevant information is included and accurately presented."
        elif confidence_score >= 0.7:
            feedback = "The explanation is detailed and technically sound, making it well-suited for an expert audience. You might consider adding even more technical depth if appropriate."

    
    return feedback



In [ ]:
import pandas as pd

df = pd.read_csv('expert_explanation_dataset.csv')
print(len(df))
new_df = df[df['confidence_score'] > 0.7]
print(len(new_df))
new_df.to_csv('fuck_you.csv', index=False)
ques = new_df.sample(1)[['explanation']]

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the fine-tuned BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained(
    'bert-finetuned-expert_high_confidence')
tokenizer = BertTokenizer.from_pretrained(
    'bert-finetuned-expert_high_confidence')


def get_confidence_score(sentence):
    model.eval()
    inputs = tokenizer(sentence, return_tensors="pt",
                       truncation=True, padding='max_length', max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    # Assuming a single-label regression task
    confidence_score = torch.sigmoid(logits).item()
    return confidence_score


# Example sentence
sentence = "Strategically, the utility function \( U_u(\omega_t^o) \) must be evaluated against the predefined quantile thresholds to ensure the offer remains within acceptable limits."
confidence_score = get_confidence_score(sentence)
print(f"Confidence Score: {confidence_score}")

In [ ]:
# Function to get user choice
def get_user_choice():
    while True:
        choice = input(
            "Choose the target audience (layperson/expert): ").strip().lower()
        if choice in ['layperson', 'expert']:
            return choice
        else:
            print("Invalid choice. Please enter 'layperson' or 'expert'.")


# Layperson threshold (you might want to adjust this if you're doing expert explanations)
threshold = 0.7

# Get the user's choice for target audience
target_audience = get_user_choice()

# Loop until explanation meets the threshold for the chosen audience
explanation = ""
feedback = ""
while True:
    explanation = custom_explanation(
        enriched_sentence, target_audience, prompt_layperson if target_audience == 'layperson' else prompt_expert, max_tokens=400, temperature=0.7, top_p=1.0,)

    # Output the generated explanation
    print(f"Generated Explanation for {target_audience}:\n{explanation}\n")

    score = validate_explanation(explanation, target_audience)

    # Output the confidence score
    print(f"Confidence Score: {score}\n")

    feedback = generate_feedback(explanation, score, target_audience)

    if score >= threshold:
        print(
            f"Final Explanation for {target_audience} based on enriched sentence:\n\n{explanation}\n")
        break
    else:
        print(f"{target_audience.capitalize()} explanation below threshold with score {score}, refining... Feedback: {feedback}")

In [ ]:
# Layperson threshold
threshold = 0.7

# Loop until explanation meets the threshold for layperson
layperson_explanation = ""
layperson_feedback = ""
while True:
    layperson_explanation = custom_explanation(
        enriched_sentence, 'layperson', prompt_layperson, max_tokens=400, temperature=0.7, top_p=0.9,)
    layperson_score = validate_explanation(
        layperson_explanation, 'layperson')
    layperson_feedback = generate_feedback(
        layperson_explanation, layperson_score, 'layperson')
    if layperson_score >= threshold:
        print(
            f"Final Explanation for Layperson based on enriched sentence:\n\n{layperson_explanation}\n")
        break
    else:
        print(
            f"Layperson explanation below threshold with score {layperson_score}, refining... Feedback: {layperson_feedback}")



In [ ]:
# Layperson threshold
threshold = 0.7

# Loop until explanation meets the threshold for layperson
layperson_explanation = ""
layperson_feedback = ""
while True:
    layperson_explanation = custom_explanation(
        enriched_sentence, 'layperson', prompt_layperson, max_tokens=400, temperature=0.7, top_p=0.9,)

    # Output the generated explanation
    print(f"Generated Explanation:\n{layperson_explanation}\n")

    layperson_score = validate_explanation(
        layperson_explanation, model, tokenizer)

    # Output the confidence score
    print(f"Confidence Score: {layperson_score}\n")

    layperson_feedback = generate_feedback(
        layperson_explanation, layperson_score, 'layperson')

    if layperson_score >= threshold:
        print(
            f"Final Explanation for Layperson based on enriched sentence:\n\n{layperson_explanation}\n")
        break
    else:
        print(
            f"Layperson explanation below threshold with score {layperson_score}, refining... Feedback: {layperson_feedback}")

In [ ]:
#Expert threshold (similar to layperson if needed)
threshold = 0.6
#Loop until explanation meets the threshold for expert
expert_explanation = ""
expert_feedback = ""
while True:
     expert_explanation = custom_explanation(
         enriched_sentence, 'expert', prompt_expert, max_tokens=400, temperature=0.7, top_p=0.9)
     expert_score = validate_explanation(expert_explanation, model, tokenizer)
     expert_feedback = generate_feedback(expert_explanation, expert_score, 'expert')
     if expert_score >= threshold:
         print(f"Final Explanation for Expert based on enriched sentence:\n\n{expert_explanation}\n")
         break
     else:
         print(f"Expert explanation below threshold with score {expert_score}, refining... Feedback: {expert_feedback}")

In [ ]:
import openai
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Initialize OpenAI API
openai.api_key = 'sk-proj-eCKWxtFXUDb0yThKFtx1T3BlbkFJXvgHaR4d455KUPYGVi4L'

# Function to generate explanation using GPT-4
def generate_gpt_explanation(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You're an expert assistant who provides clear and concise explanations"},
            {"role": "user", "content": prompt}
        ],
        max_tokens=200,
        temperature=0.7,
        top_p=0.9,
        frequency_penalty=0.0,
        presence_penalty=0.0,
    )
    explanation = response['choices'][0]['message']['content'].strip()
    return explanation

# Function to validate explanation using BERT
def validate_explanation_with_bert(explanation, model, tokenizer, max_length=512):
    model.eval()
    inputs = tokenizer(explanation, return_tensors="pt", truncation=True, padding='max_length', max_length=max_length)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    confidence_score = torch.sigmoid(logits).item()
    return confidence_score

# Load fine-tuned BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-finetuned-explanation')
tokenizer = BertTokenizer.from_pretrained('bert-finetuned-explanation')

# Example mathematical sentence and prompts
sentence = "Ensures \(U_u(\omega_t^o) \) meets either a calculated statistical value or a specified minimum utility requirement in the initial interval \( [0.000, 0.0361) \)"
prompt = (
    f"Explain the following mathematical statement in very simple terms, suitable for a layperson. The explanation should be within 50 words. Use the examples below as a guide:\n\n"
    f"**Mathematical Statement:**\n{sentence}\n\n"
    "**Examples for Layperson:**\n"
    "1. In the first time period, the value of opponent's offer must be at least as large as either a calculated value from the data or a set minimum value.\n"
    "2. In the second time period, the value of the opponent's offer has to be at least as big as either a set minimum value or another calculated value from the data.\n"
    "3. At the start, the menu variety must be as extensive as either a basic selection or a preferred choice.\n"
    "4. At the start of planning, the budget must be at least as large as either the estimated cost or a set limit.\n\n"
    "Provide a similar style explanation suitable for a layperson, within 50 words."
)

# Generate explanation using GPT-4
gpt_explanation = generate_gpt_explanation(prompt)
print(f"GPT-4 Explanation for Layperson:\n{gpt_explanation}\n")

# Validate explanation using BERT
confidence_score = validate_explanation_with_bert(gpt_explanation, model, tokenizer)
print(f"Confidence Score by BERT: {confidence_score}\n")
